# Basic tutorial: translation
#### Author: Matteo Caorsi

This short tutorial provides you with the basic functioning of *giotto-deep* API.

The example described in this tutorial is the one of translation.

The main steps of the tutorial are the following:
 1. creation of a dataset
 2. creation of a model
 3. define metrics and losses
 4. train the model
 5. extract some features of the network

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np

import torch
from torch import nn

from gdeep.models import FFNet

from gdeep.visualisation import  persistence_diagrams_of_activations

from torch.utils.tensorboard import SummaryWriter
from gdeep.data.datasets import TorchDataLoader
from gdeep.pipeline import Pipeline

from gtda.diagrams import BettiCurve

from gtda.plotting import plot_betti_surfaces

# Initialize the tensorboard writer

In order to analyse the reuslts of your models, you need to start tensorboard.
On the terminal, move inside the `/example` folder. There run the following command:

```
tensorboard --logdir=runs
```

Then go [here](http://localhost:6006/) after the training to see all the visualisation results.

In [ ]:
writer = SummaryWriter()

# Create your dataset

In [ ]:
from torch.utils.data.sampler import SubsetRandomSampler

# the only part of the training set we are interested in
train_indices = list(range(32*10))

dl = TorchDataLoader(name="Multi30k", convert_to_map_dataset=True)
dl_tr_str, dl_ts_str = dl.build_dataloaders(sampler=SubsetRandomSampler(train_indices))


The dataset contains a list of pairs of sentences: the English sentecen and its German translation.

In [ ]:
print("Before preprocessing: \n", dl_tr_str.dataset[0])

## Required preprocessing

Neural networks cannot direcly deal with strings. We have first to preprocess the dataset in three main ways:
 1. Tokenise the string into its words
 2. Build a vocabulary out of these words
 3. Embed each word into a vector, so that each sentence becomes a list of vectors

The first two steps are performed by the `PreprocessTextTranslation`. The embedding will be added directly to the model.

In [ ]:
from gdeep.data import TransformingDataset
from gdeep.data.preprocessors import TokenizerTranslation
from gdeep.data.datasets import BuildDataLoaders

tokenizer = TokenizerTranslation()
textds = dl_tr_str.dataset

tokenizer.fit_to_dataset(textds)

transformed_textds = tokenizer.attach_transform_to_dataset(dl_tr_str.dataset)

transformed_textts = tokenizer.attach_transform_to_dataset(dl_ts_str.dataset)

print("After the preprocessing: \n", transformed_textds[0])

train_indices = list(range(64*2))
test_indices = list(range(64*1))


dl_tr, dl_ts, _ = BuildDataLoaders((transformed_textds, 
                                   transformed_textts)).build_dataloaders(batch_size=16, 
                                                                          sampler=SubsetRandomSampler(train_indices))




## Define and train your model

In [ ]:
from torch.nn import Transformer
from torch.optim import Adam, SparseAdam, SGD
import copy

# my simple transformer model
class TranslationTransformer(nn.Module):

    def __init__(self, src_vocab_size, tgt_vocab_size, embed_dim):
        super(TranslationTransformer, self).__init__()
        self.transformer = Transformer(d_model=embed_dim,
                                       nhead=2,
                                       num_encoder_layers=1,
                                       num_decoder_layers=1,
                                       dim_feedforward=512,
                                       dropout=0.1)
        self.embedding_src = nn.Embedding(src_vocab_size, embed_dim, sparse=True)
        self.embedding_tgt = nn.Embedding(tgt_vocab_size, embed_dim, sparse=True)
        self.generator = nn.Linear(embed_dim, tgt_vocab_size)
        
    def forward(self, X):
        #print(X.shape)
        src = X[:,0,:]
        tgt = X[:,1,:]
        #print(src.shape, tgt.shape)
        src_emb = self.embedding_src(src)
        tgt_emb = self.embedding_tgt(tgt)
        #print(src_emb.shape, tgt_emb.shape)
        self.outs = self.transformer(src_emb, tgt_emb)
        #print(outs.shape)
        logits = self.generator(self.outs)
        #print(logits.shape)
        #out = torch.topk(logits, k=1, dim=2).indices.reshape(-1,44)
        #print(out, out.shape)
        return logits
    
    def __deepcopy__(self, memo):
        """this is needed to make sure that the 
        non-leaf nodes do not
        interfere with copy.deepcopy()
        """
        cls = self.__class__
        result = cls.__new__(cls)
        memo[id(self)] = result
        for k, v in self.__dict__.items():
            setattr(result, k, copy.deepcopy(v, memo))
        return result
    
    def encode(self, src, src_mask):
        """this method is used only at the inference step"""
        return self.transformer.encoder(
                            self.embedding_src(src), src_mask)

    def decode(self, tgt, memory, tgt_mask):
        """this method is used only at the inference step"""
        return self.transformer.decoder(
                          self.embedding_tgt(tgt), memory,
                          tgt_mask)
    


In [ ]:
vocab_size = 50000 # to be discussed

src_vocab_size = vocab_size # len(prec.vocabulary)
tgt_vocab_size = vocab_size # len(prec.vocabulary_target)
emb_dim = 64

model = TranslationTransformer(src_vocab_size, tgt_vocab_size, emb_dim)
X = next(iter(dl_tr));
#assert model(X[0]).argmax(2).shape == X[1].shape
print(model)

## Define the loss function

This loss function is a adapted version of the Cross Entropy for the trnasformer architecture.

In [ ]:

def loss_fn(logits, tgt_out):
    cel = nn.CrossEntropyLoss()
    return cel(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))


In [ ]:
# prepare a pipeline class with the model, dataloaders loss_fn and tensorboard writer
pipe = Pipeline(model, (dl_tr, dl_ts), loss_fn, writer)

# train the model
pipe.train(SGD, 3, False, {"lr":0.01}, {"batch_size":16})

## Translation!

In [ ]:
de, en = next(iter(dl_tr_str))
print(de, "\n", en)


Get the vocabulary and numericize the German sentence

In [ ]:
voc = tokenizer.vocabulary_target
sent = str.lower(de[0]).split()
de_sentence = list(map(voc.__getitem__,sent))
de_sentence

In [ ]:

def greedy_decode(model, src, src_mask, max_len, start_symbol):
    """function to generate output sequence using greedy algorithm"""
    memory = model.encode(src, src_mask)
    out = model.decode(src, memory, None)
    prob = model.generator(out)
    greedy_out = torch.max(prob, dim=2).indices
    return greedy_out
    

def translate(model: torch.nn.Module, dl_ts_item):
    """actual function to translate input sentence into target language"""
    model.eval()
    voc = tokenizer.vocabulary_target
    src = dl_ts_item
    num_tokens = src.shape[1]
    src_mask = None
    tgt_tokens_raw = greedy_decode(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=111).flatten()
    tgt_tokens = [token for token in tgt_tokens_raw if token < len(list(voc.vocab))]
    return " ".join(list(map(list(voc.vocab).__getitem__, tgt_tokens)))
    
# translation!
print("German sentence: ", de[0])
print("English translation: ", translate(pipe.model, torch.tensor([de_sentence])))

# Extract inner data from your models

In [ ]:
from gdeep.models import ModelExtractor

me = ModelExtractor(pipe.model, loss_fn)

lista = me.get_layers_param()

for k, item in lista.items():
    print(k,item.shape)


In [ ]:
DEVICE = torch.device("cpu")
x = next(iter(dl_tr))[0]
pipe.model.eval()
pipe.model(x.to(DEVICE))

list_activations = me.get_activations(x)
len(list_activations)


In [ ]:
x = next(iter(dl_tr))[0][0]
if x.dtype is not torch.int64:
    res = me.get_decision_boundary(x, n_epochs=1)
    res.shape

In [ ]:
x, target = next(iter(dl_tr))
if x.dtype is torch.float:
    for gradient in me.get_gradients(x, target=target)[1]:
        print(gradient.shape)

# Visualise activations and other topological aspects of your model

In [ ]:
from gdeep.visualisation import Visualiser

vs = Visualiser(pipe)

vs.plot_data_model()
#vs.plot_activations(x)
#vs.plot_persistence_diagrams(x)
